### classify stackoverflow posts by considering each document a simple mean of its individual word embeddings

**TODO**
 - use weighted averages (tf-id, ppmi, etc)
 - use a different scoring function so that wrong guesses that are close to the correct values (think "python", "python-2") are penalized less than guesses that are wildly wrong.


In [1]:
import csv
import os
import re
import sys
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

# my stuff in the helpers/ directory
from helpers import files, texts, metrics, tags

Using TensorFlow backend.


In [2]:
np.random.seed(seed=42)

In [3]:
texts, labels = files.read_stackoverflow_sample_small()

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(len(texts)/3000)

In [5]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [6]:
truncated_labels = tags.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [7]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(labels)

In [8]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags.get_probabilities_index(binary_labels)

In [9]:
# word => embedding
embeddings_index = files.read_glove_100()

In [10]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((22000, 1000), (22000, 9022))

In [11]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(GlobalAvgPool1D())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')
# multi label needs a specific accuracy function

model.fit(X_train, Y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_val,Y_val))

Train on 17600 samples, validate on 4400 samples
Epoch 1/10
17600/17600 [==============================] - 5s - loss: 0.0941 - val_loss: 0.0044
Epoch 2/10
17600/17600 [==============================] - 5s - loss: 0.0040 - val_loss: 0.0032
Epoch 3/10
17600/17600 [==============================] - 4s - loss: 0.0033 - val_loss: 0.0030
Epoch 4/10
17600/17600 [==============================] - 4s - loss: 0.0031 - val_loss: 0.0029
Epoch 5/10
17600/17600 [==============================] - 4s - loss: 0.0030 - val_loss: 0.0028
Epoch 6/10
17600/17600 [==============================] - 4s - loss: 0.0029 - val_loss: 0.0027
Epoch 7/10
17600/17600 [==============================] - 5s - loss: 0.0028 - val_loss: 0.0027
Epoch 8/10
17600/17600 [==============================] - 5s - loss: 0.0028 - val_loss: 0.0026
Epoch 9/10
17600/17600 [==============================] - 5s - loss: 0.0027 - val_loss: 0.0025
Epoch 10/10
17600/17600 [==============================] - 5s - loss: 0.0026 - val_loss: 0.0025


In [26]:
# sample result for a couple of test cases
num_test_cases = 10
THRESHOLD = 70.0

relative_difference_threshold = THRESHOLD

for i in np.random.randint(low=0, high=len(Y_val), size=num_test_cases):

    actual_label_indices = Y_val[i].reshape(1,-1)
    predicted_tag_probabilities = model.predict(X_val[i].reshape(1,-1))
    
    predicted_label_indices = tags.get_predicted_indices(tag_probabilities_index,
                                                        predicted_tag_probabilities,
                                                        relative_difference_threshold)

    actual_labels = lb.inverse_transform(actual_label_indices)
    
    predicted_label_scores = predicted_tag_probabilities[0][predicted_label_indices.ravel().nonzero()[0]]
    predicted_labels = lb.inverse_transform(predicted_label_indices)

    tags_and_scores = sorted(list(zip(predicted_labels[0],predicted_label_scores)), key=lambda tpl: tpl[1], reverse=True)
    
    print(actual_labels)
    print(tags_and_scores)
    print('\n\n')


[('java', 'jboss', 'memory-leaks', 'rhel6')]
[]



[('android', 'bind', 'music', 'player', 'service')]
[]



[('angularjs',)]
[('api', 0.005911795), ('cocoa', 0.0053084549), ('flash', 0.0052143522), ('facebook-graph-api', 0.0049816384), ('github', 0.0044513606), ('cocoa-touch', 0.0044459733), ('pdf', 0.0039695352), ('vim', 0.0038637891), ('testing', 0.0036391772), ('silverlight', 0.0035436652), ('elasticsearch', 0.003523462), ('svn', 0.0034538943), ('bitmap', 0.0033563231), ('javafx', 0.0032467349), ('intellij-idea', 0.0032130079), ('sharepoint', 0.0031792934), ('windows-phone-7', 0.003114945), ('yii', 0.0031024697), ('tfs', 0.002972536), ('build', 0.0029003997), ('selenium-webdriver', 0.0028565382), ('google-chrome-extension', 0.0028014011), ('logging', 0.0026500104), ('emacs', 0.0026242412), ('plugins', 0.0026189734), ('paypal', 0.0026157321), ('mobile', 0.00254815), ('web', 0.0025309101), ('timer', 0.0025243929), ('boost', 0.0025013182), ('charts', 0.0024808734), ('visual-studio-201

[('postgresql', 'xml')]
[('api', 0.0065941308), ('flash', 0.0060473788), ('cocoa', 0.0060148886), ('delphi', 0.0059678056), ('facebook-graph-api', 0.0058399136), ('github', 0.0053289514), ('cocoa-touch', 0.0050736545), ('ipad', 0.0049189799), ('actionscript-3', 0.0046986239), ('vim', 0.0046758093), ('variables', 0.004470489), ('pdf', 0.0043776706), ('url', 0.0041478882), ('c#-4.0', 0.0041416371), ('jquery-ui', 0.0041412064), ('elasticsearch', 0.0041391538), ('visual-studio-2012', 0.0041133794), ('bitmap', 0.0040723011), ('internet-explorer', 0.0040669688), ('silverlight', 0.0040638568), ('testing', 0.0040637623), ('svn', 0.0040400359), ('dom', 0.0040156581), ('intellij-idea', 0.0039654556), ('docker', 0.0038834137), ('windows-phone-7', 0.0038100102), ('sharepoint', 0.0037409093), ('yii', 0.0037043639), ('javafx', 0.0037013751), ('command-line', 0.0036105008), ('build', 0.003512034), ('tfs', 0.0034568436), ('selenium-webdriver', 0.0034209932), ('android-activity', 0.0034121675), ('googl

In [ ]:
X_test_list = []

for validation_features in X_val:
    X_test_list.append(validation_features.reshape(1,-1))

X_test = np.vstack(X_test_list)    

In [ ]:
Y_pred_lst = [model.predict(x_test.reshape(1,-1)) for x_test in X_test]

Y_pred_calculated_lst = [tags.get_predicted_indices(tag_probabilities_index,y_pred,THRESHOLD) for y_pred in np.vstack(Y_pred_lst)]

Y_pred = np.vstack(Y_pred_calculated_lst)

Y_val.shape,Y_pred.shape

In [ ]:
micro_f1 = metrics.calculate_multilabel_metrics(Y_val,Y_pred)
print('micro-averaged F1 score (validation set) is {0}'.format(micro_f1))

micro_prec = metrics.calculate_multilabel_metrics(Y_val, Y_pred, metric='precision')
print('micro-averaged precision score (validation set) is {0}'.format(micro_prec))

micro_rec = metrics.calculate_multilabel_metrics(Y_val, Y_pred, metric='recall')
print('micro-averaged recall score (validation set) is {0}'.format(micro_rec))
